In [4]:
# 调用Deepseek-R1示例，支持流式输出和思考链
from openai import OpenAI

client = OpenAI(
	base_url="https://ai.gitee.com/v1",
	api_key="DUQFR61KA8QLDVEQPGJKBXYSL2DXMPST1FM98Y1L",
	default_headers={"X-Failover-Enabled":"true"},
)

response = client.chat.completions.create(
	messages=[
		{
			"role": "system",
			"content": "You are a helpful and harmless assistant. You should think step-by-step."
		},
		{
			"role": "user",
			"content": "Can you please let us know more details about your "
		}
	],
	model="DeepSeek-R1",
	stream=True,
	max_tokens=8192,
	temperature=0.6,
	top_p=0.7,
	extra_body={
		"top_k": 50,
	},
	frequency_penalty=0,
)

fullResponse = ""
print("Response:")
# Print streaming response
for chunk in response:
	if len(chunk.choices) == 0:
		continue
	delta = chunk.choices[0].delta
	# If is thinking content, print it in gray
	if hasattr(delta, 'reasoning_content') and delta.reasoning_content:
		fullResponse += delta.reasoning_content
		print(f"\033[90m{delta.reasoning_content}\033[0m", end="", flush=True)
	elif delta.content:
		fullResponse += delta.content
		print(delta.content, end="", flush=True)

Response:
<think>
Okay, the user's query is incomplete: "Can you please let us know more details about your..." - it cuts off mid-sentence. Interesting. They're probably asking about my capabilities or specifications but didn't finish typing.  

Hmm, looking at the context, this seems like a fresh interaction since the assistant's initial greeting is the only prior message. The user might be new to interacting with me or testing my responses. Since they're asking about "your," they likely want to understand what I can do before diving deeper.  

I should assume positive intent—they're probably curious, not malicious. Best approach is to politely point out the incomplete query while proactively offering help. Listing common topics users ask about (capabilities, limitations, etc.) covers likely intentions without making them rephrase.  

The tone needs to stay warm and encouraging ("happy to help!"). Adding emojis (👋😊) keeps it friendly since the user might feel awkward about the typo. E

# 1、调用API

In [ ]:
# --- API ---
from openai import OpenAI

client = OpenAI(
    base_url="https://ai.gitee.com/v1",
    api_key="xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx",
    default_headers={"X-Failover-Enabled":"true"},
)

# 2、加载处理的prompt.txt

In [2]:
# Load prompt from text file
def load_text(path: str) -> str:
    with open(path, "r", encoding="utf-8") as f:
        return f.read()

SYSTEM_PROMPT = load_text("law_prompt.txt")  # 这里用你的prompt .txt

# 3、抽取知识的关系并输出json文件

In [4]:
# 抽取知识图谱
import re
import json
# 从一大串字符串中“抠出”最外层 JSON
def extract_json_from_string(s: str) -> str:
    """
    在字符串中找到第一个 { 到 最后一个 } 之间的内容，
    认为这是 JSON 对象，返回这段子串。
    """
    s = s.strip()
    # 用正则搜索最外层的大括号
    m = re.search(r'\{.*\}', s, re.S)
    if not m:
        raise ValueError("在模型输出中没有找到 JSON 对象，原始内容为：\n" + s)
    return m.group(0)

# 4. 用流式的方式调用 DeepSeek-R1，抽取知识图谱
def extract_kg_stream(text: str):
    response = client.chat.completions.create(
        model="DeepSeek-R1",
        stream=True,                      # ✅ 保留流式
        response_format={"type": "json_object"},  # ✅ 告诉服务端我们要 JSON
        max_tokens=1500,
        temperature=0.2,
        top_p=0.7,
        extra_body={
            "top_k": 50,
            "enable_reasoning": False,     # ✅ 关闭长时间推理，避免一直跑
        },
        messages=[
            {
                "role": "system",
                "content": SYSTEM_PROMPT
            },
            {
                "role": "user",
                "content": f"请从以下文本中抽取知识图谱实体和关系：\n\n{text}"
            }
        ],
    )

    full_json_str = ""
    print("Response:")

    # 5. 流式逐块接收
    for chunk in response:
        if len(chunk.choices) == 0:
            continue

        delta = chunk.choices[0].delta

        # 如果有推理内容（reasoning_content），只打印，不参与 JSON
        if hasattr(delta, "reasoning_content") and delta.reasoning_content:
            # 灰色输出思考过程，仅供你调试看
            print(f"\033[90m{delta.reasoning_content}\033[0m", end="", flush=True)

        # 真正的 JSON 字符串在 content 里
        elif delta.content:
            full_json_str += delta.content
            # 你也可以边打印边看模型在输出什么
            print(delta.content, end="", flush=True)

    print("\n\n=== Raw streamed content ===")
    print(full_json_str)

    # 6. 从整串内容里抠出 JSON 部分
    json_str = extract_json_from_string(full_json_str)
    print("\n=== Extracted JSON string ===")
    print(json_str)

    # 7. 解析为 Python dict
    kg_data = json.loads(json_str)
    print("\n✅ JSON 解析成功！")
    return kg_data

In [5]:
# 测试文本
test_text = """
第十条 著作权包括下列权利：
（一）发表权，即决定作品是否公之于众的权利；
（二）署名权，即表明作者身份，在作品上署名的权利；
（三）修改权，即修改或者授权他人修改作品的权利；
（四）保护作品完整权。
"""

kg = extract_kg_stream(test_text) # 抽取知识图谱

print("\n=== 最终解析结果（美化输出） ===")
print(json.dumps(kg, ensure_ascii=False, indent=2))

Response:
首先，用户提供了一段文本："第十条 著作权包括下列权利：（一）发表权，即决定作品是否公之于众的权利；（二）署名权，即表明作者身份，在作品上署名的权利；（三）修改权，即修改或者授权他人修改作品的权利；（四）保护作品完整权。"

我的任务是提取实体和关系，构建知识图谱。输出必须是JSON格式，放在`content`字段中，没有任何解释。任何推理步骤放在`reasoning_content`中。

输出格式：
- `entities`: 列表，每个实体有 `uid`, `node_type`, `name`, `extra`（可选）。
- `relations`: 列表，每个关系有 `from_uid`, `to_uid`, `rel_type`。

规则：
- `uid` 必须唯一且稳定。我应该基于内容生成稳定的标识符。
- `node_type` 必须是其中之一：["Law","Chapter","Article","Right","Concept","Definition","Case","Entity"]。
- `rel_type` 必须大写且简洁。
- `extra` 是可选的键值对信息。

从文本中识别实体：
- 文本是"第十条"，这很可能是一个Article（条款）。在之前的例子中，"第十条" 被映射到 "Article:第10条"。
- 文本提到"著作权"，这似乎是著作权的定义或权利列表。
- 权利包括：发表权、署名权、修改权、保护作品完整权。这些应该是权利实体。

实体提取：
1. **Article**: "第十条"，uid 应该类似 "Article:第10条"。name 可以是 "第十条"。
2. **Law**: 文本中没有直接提到法律名称，但上下文是关于著作权的，所以在之前的例子中有 "Law:著作权法"。用户提供的文本是独立的，我应该基于内容推断。文本开始就是"第十条"，暗示它是著作权法的一部分。我应该假设这个Article属于著作权法。所以，我需要一个 Law 实体。uid "Law:著作权法"，name "中华人民共和国著作权法"，如示例所示。
3. **Rights**: 每个权利：发表权、署名权、修改权、保护作品完整权。这些应该是节点类型 "Right"，因为列表中是权利。

节点类型选项：["Law","Chap

# 4、保存生成的JSON文件

In [6]:
import os
import json

# 你想保存 JSON 的目录（改成你自己的）
OUTPUT_DIR = "D:\\Desktop\\现代软件工程\\作业\\实践营\\KG\\KG_files\\KG_json_test"
# 如果目录不存在就创建
os.makedirs(OUTPUT_DIR, exist_ok=True)


In [7]:
def save_kg_json(kg: dict, output_dir: str, filename: str):
    """把 kg（Python 字典）保存为一个 .json 文件"""
    path = os.path.join(output_dir, filename)
    with open(path, "w", encoding="utf-8") as f:
        json.dump(kg, f, ensure_ascii=False, indent=2)
    print(f"已保存到: {path}")

In [8]:
# 从 kg 里取出条文 uid 或 name，当作文件名一部分
article_uid = None
for ent in kg["content"]["entities"]:
    if ent["node_type"] == "Article":
        article_uid = ent["uid"]  # 比如 "Article:第10条"
        break

if article_uid is None:
    filename = "unknown_article.json"
else:
    # 简单处理下文件名里的冒号
    safe_uid = article_uid.replace(":", "_")
    filename = f"{safe_uid}.json"   # Article_第10条.json

save_kg_json(kg, OUTPUT_DIR, filename)

已保存到: D:\Desktop\现代软件工程\作业\实践营\KG\KG_files\KG_json_test\Article_第10条.json


# 5、连接到Neo4j数据库

In [9]:
# 连接到Neo4j数据库
from neo4j import GraphDatabase

NEO4J_URI = "neo4j://127.0.0.1:7687"
NEO4J_USER = "neo4j"
NEO4J_PASSWORD = "20011127"  # 换成你 Neo4j Desktop 里设置的

driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))
print("Neo4j driver initialized.")

Neo4j driver initialized.


In [10]:
# 检查连接是否成功
from neo4j import GraphDatabase

driver = GraphDatabase.driver("bolt://localhost:7687", auth=("neo4j", "20011127"))
with driver.session() as session:
    result = session.run("RETURN 1 AS test")
    print(result.single())

<Record test=1>


In [11]:
# 确保每个 KGNode 的 uid 唯一（只需要跑一次）
with driver.session() as session:
    session.run("""
    CREATE CONSTRAINT kgnode_uid_unique IF NOT EXISTS
    FOR (n:KGNode)
    REQUIRE n.uid IS UNIQUE
    """)
print("Constraint created (if it didn't exist).")

# 写入实体
def upsert_entities(tx, entities):
    """
    把大模型返回的 entities 写进 Neo4j。
    每个实体对应一个 :KGNode 节点，uid 作为唯一标识。
    结构假定为：
    {
      "uid": "Law:著作权法",
      "node_type": "Law",
      "name": "中华人民共和国著作权法",
      "extra": {...}
    }
    """
    query = """
    UNWIND $entities AS ent
    MERGE (n:KGNode {uid: ent.uid})
    SET n.node_type = ent.node_type,
        n.name      = ent.name
    SET n += ent.extra
    """
    tx.run(query, entities=entities)

# 写入关系
def upsert_relations(tx, relations):
    """
    把 relations 写成 Neo4j 里的关系。
    关系结构假定为：
    {
      "from_uid": "Law:著作权法",
      "to_uid":   "Article:第10条",
      "rel_type": "HAS_ARTICLE"
    }
    这里统一用关系类型 :REL，具体类型存在 r.rel_type 属性里。
    """
    query = """
    UNWIND $relations AS rel
    MATCH (a:KGNode {uid: rel.from_uid})
    MATCH (b:KGNode {uid: rel.to_uid})
    MERGE (a)-[r:REL {rel_type: rel.rel_type}]->(b)
    """
    tx.run(query, relations=relations)

print("upsert_entities & upsert_relations 已定义。")

Constraint created (if it didn't exist).
upsert_entities & upsert_relations 已定义。


In [12]:
entities = kg["content"]["entities"]
relations = kg["content"]["relations"]

with driver.session() as session:
    session.execute_write(upsert_entities, entities)
    session.execute_write(upsert_relations, relations)

print("当前这条法条已写入 Neo4j。")

当前这条法条已写入 Neo4j。


# 读取目录下的所有json文件

In [13]:
# 读取保存的json文件
import os
import json
JSON_DIR = "D:\\Desktop\\现代软件工程\\作业\\实践营\\KG\\KG_files\\KG_json_test"  # ← 你的目录

In [15]:
def import_json_to_neo4j(json_dir):
    files = [f for f in os.listdir(json_dir) if f.endswith(".json")]
    print("检测到 JSON 文件：", files)

    for filename in files:
        path = os.path.join(json_dir, filename)
        print(f"\n>>> 处理文件：{path}")

        with open(path, "r", encoding="utf-8") as f:
            kg = json.load(f)

        entities = kg["content"]["entities"]
        relations = kg["content"]["relations"]

        with driver.session() as session:
            session.execute_write(upsert_entities, entities)
            session.execute_write(upsert_relations, relations)

    print("\n🎉 所有 JSON 文件已成功写入 Neo4j！")

In [17]:
import_json_to_neo4j(JSON_DIR)

检测到 JSON 文件： ['Article_第10条.json']

>>> 处理文件：D:\Desktop\现代软件工程\作业\实践营\KG\KG_files\KG_json_test\Article_第10条.json

🎉 所有 JSON 文件已成功写入 Neo4j！
